In [1]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time

In [2]:
"""
def generate_H_S_CNOT_T_circuit(qubits, gates,p_t=0,seed=1000):
    random.seed(seed)
    p_s = 0.333*(1.0-p_t)
    p_had = 0.333*(1.0-p_t)
    p_cnot = 1.0-p_s-p_had-p_t

    c = zx.Circuit(qubits)
    for _ in range(gates):
        r = random.random()
        if r > 1-p_had:
            c.add_gate("HAD",random.randrange(qubits))
        elif r > 1-p_had-p_s:
            c.add_gate("S",random.randrange(qubits))
        elif r > 1-p_had-p_s-p_t:
            c.add_gate("T",random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl!=tgt: break
            c.add_gate("CNOT",tgt,ctrl)
    return c

"""    

'\ndef generate_H_S_CNOT_T_circuit(qubits, gates,p_t=0,seed=1000):\n    random.seed(seed)\n    p_s = 0.333*(1.0-p_t)\n    p_had = 0.333*(1.0-p_t)\n    p_cnot = 1.0-p_s-p_had-p_t\n\n    c = zx.Circuit(qubits)\n    for _ in range(gates):\n        r = random.random()\n        if r > 1-p_had:\n            c.add_gate("HAD",random.randrange(qubits))\n        elif r > 1-p_had-p_s:\n            c.add_gate("S",random.randrange(qubits))\n        elif r > 1-p_had-p_s-p_t:\n            c.add_gate("T",random.randrange(qubits))\n        else:\n            tgt = random.randrange(qubits)\n            while True:\n                ctrl = random.randrange(qubits)\n                if ctrl!=tgt: break\n            c.add_gate("CNOT",tgt,ctrl)\n    return c\n\n'

In [3]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [4]:
# seed固定
seed = 1000
random.seed(seed)

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [100, 500, 1000, 2000, 5000, 10000, 50000,100000]

# CSV書き込みの準備
output_file = "./kekka/jisaku_datasets.csv"
fieldnames = [
    "data_name","qubits", "gates", "p_t","seeds","depth","tcount","H","S","CNOT","time"
]

with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()


    for qubit in qubits_list:
        for gate_num in gate_num_lists:
            for p_t in p_t_lists:
                try:
                    start = time.time()
                    # 回路作成
                    c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)
                    #circuit_stats = c.stats_dict()

                    #qiskit変換
                    circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())

                    depth = circuit_qiskit.depth()
                    gate_count_dict = circuit_qiskit.count_ops()
                    #print("cx",gate_count_dict["cx"] , "h",gate_count_dict["h"], "s",gate_count_dict["s"], "t",gate_count_dict["t"])

                    end = time.time()

                    #dataname = "q" + str(qubit) + "_gates" + str(gate_num) + "_pt" +str(p_t)+ "_seeds" + str(seed)
                    # 結果をCSVに書き込む
                    writer.writerow({
                        "data_name": "q" + str(qubit) + "_gates" + str(gate_num) + "_pt" +str(p_t)+ "_seeds" + str(seed),
                        "qubits": qubit,
                        "gates": gate_num,
                        "p_t": p_t,
                        "seeds": seed,
                        "depth": depth,
                        "tcount": gate_count_dict.get("t","0"),
                        "H": gate_count_dict["h"],
                        "S": gate_count_dict["s"],
                        "CNOT": gate_count_dict["cx"],
                        "time": end-start
                    })

                    print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                except Exception as e:
                    print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

Error for Qubits=4, Gate_Num=100, P_T=0.0: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.04: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.08: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.12: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.16: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.2: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.24: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.28: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.32: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.36: 'collections.OrderedDict' object is not callable
Error for Qubits=4, Gate_Num=100, P_T=0.4: 'collections.OrderedDict' obj